## GROUPING METRICS
Get all the metrics together in the same dataframe

In [15]:
import os
import pandas as pd
import numpy as np
import ipywidgets as widgets
from bqplot import *
import json
pd.options.display.max_columns = 999

In [42]:
# load tracking data
current_directory = os.getcwd()
path_tracking = os.path.join(os.path.join(os.path.dirname(current_directory),'data'),"tracking_set_0")
print(path_tracking)
game_id = 1

df_tracking = pd.read_parquet(f'{path_tracking}/{game_id}_tracking.parquet')

#           ------------------------------------------------------------        

# load events names
path_event_csv = os.path.join(os.path.dirname(current_directory),'data')
df_event_names = pd.read_csv(os.path.join(path_event_csv,'event_names.csv'))
dict_event_names = df_event_names.set_index('event_type_id').to_dict()['event_description']

# load event data
def load_event_data(file_name, base_path):
    # read in event file
    with open(f'{base_path}/{file_name}') as f:
        data=json.loads(f.read())

    f.close()
    
    # transform data into pandas dataframe
    df_events = pd.json_normalize(data['liveData']['event'])
    
    # preprocess event data and keep relevant information only

    # add timeelapsed to each event
    df_events['timestamp'] = pd.to_datetime(df_events.timeStamp).apply(lambda x: x.timestamp())

    df_events = df_events.query('periodId in [1,2]')

    def add_timeelapsed_to_events(df):
        start_time = df.query('typeId==32')['timestamp'].iloc[0]
        df['timestamp_new'] = np.int64((df['timestamp'] - start_time)*1000)

        df['timeelapsed'] = df['timestamp_new'].apply(lambda x: (40 * round(x/40))/1000)

        return df

    df_events = df_events.groupby('periodId').apply(add_timeelapsed_to_events)

    df_events = df_events.drop(columns=['timeStamp','timestamp','timestamp_new'])
    
    # rename some columns
    df_events = df_events.rename(columns=
        {
            'periodId':'current_phase',
            'typeId':'event_type_id',
            'timeMin':'period_minute',
            'timeSec':'period_second'
        }
    )
    
    return df_events

path_events = os.path.join(os.path.join(os.path.dirname(current_directory),'data'),"first_10_events")
print(path_events)

event_file = f'{game_id}.json'

df_events = load_event_data(
    base_path=path_events,
    file_name=event_file
)

# add event descriptions
df_events['event_description'] = df_events['event_type_id'].map(dict_event_names)

# make a copy of it for later usage
events_all = df_events.copy()

display(df_events.head())
print(df_events['event_description'].unique())


#           ------------------------------------------------------------        


# read in qualifier list
path_data = os.path.join(os.path.dirname(current_directory),'data')
qualifier_names = pd.read_csv(os.path.join(path_data,"qualifier_names.csv"))

# explode coverts each element in each list to a separate row
cols = ['id', 'qualifier']
qualifiers = events_all[cols].explode('qualifier')
display(qualifiers.head())

print("------------")

qualifiers = qualifiers[qualifiers.qualifier.notna()].reset_index(drop=True)
print(qualifiers.shape)
print("------------")
display(qualifiers.head())
print("------------")

# save corresponding event ids for each qualifier
event_ids = qualifiers.id.tolist()

qualifiers = pd.json_normalize(qualifiers[qualifiers.qualifier.notna()]['qualifier'])
print(qualifiers.shape)
print("------------")
display(qualifiers.head())
print("------------")

qualifiers['event_id'] = event_ids
display(qualifiers.head())
print("------------")
qualifiers = qualifiers.merge(qualifier_names, how='left', on='qualifierId')
display(qualifiers.head())

c:\Users\Gabriel\OneDrive\Escritorio\SportsAnalyticsCourse\OptaForum\OptaChallenge_Clustering_Player_Styles\data\tracking_set_0
c:\Users\Gabriel\OneDrive\Escritorio\SportsAnalyticsCourse\OptaForum\OptaChallenge_Clustering_Player_Styles\data\first_10_events


C:\Users\Gabriel\AppData\Local\Temp\ipykernel_8320\513356439.py:42: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_events = df_events.groupby('periodId').apply(add_timeelapsed_to_events)


,id,eventId,event_type_id,current_phase,period_minute,period_second,contestantId,outcome,x,y,lastModified,qualifier,playerId,lineBreakingPass.linesBroken.value,passOption.player,passTarget.player,xThreat.applied,lineBreakingPass.lastLineBroken.value,pressure.pressureReceived.value,pressure.player,xThreat.removed,keyPass,assist,timeelapsed,event_description
2,2423549045,2,32,1,0,0,3c3jcs7vc1t6vz5lev162jyv7,1,0.0,0.0,2022-05-22T03:17:52Z,"[{'id': 3586084711, 'qualifierId': 127, 'value...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,Period start
3,2423549041,2,32,1,0,0,bx0cdmzr2gwr70ez72dorx82p,1,0.0,0.0,2022-05-21T18:59:34Z,"[{'id': 3586084701, 'qualifierId': 127, 'value...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,Period start
4,2423549063,3,1,1,0,0,bx0cdmzr2gwr70ez72dorx82p,1,49.9,50.0,2022-05-22T03:34:41Z,"[{'id': 3586084825, 'qualifierId': 56, 'value'...",6u2ob6fv950r1qve8uejkq2uh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.04,Pass
5,2423549097,4,1,1,0,2,bx0cdmzr2gwr70ez72dorx82p,1,31.5,57.2,2022-05-22T06:37:07Z,"[{'id': 3586085043, 'qualifierId': 213, 'value...",azuc3tma44xyrbgf5y279o1xx,0,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...","[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",0.0029771626,NaN,NaN,NaN,NaN,NaN,NaN,2.84,Pass
6,2423549113,5,1,1,0,7,bx0cdmzr2gwr70ez72dorx82p,1,49.2,95.4,2022-05-22T06:37:06Z,"[{'id': 3586085129, 'qualifierId': 212, 'value...",7sep6mx2s67mh5fr3raxu7aei,1,"[{'playerId': '5qgc6zjc38a5xjl35gs7h3vu1', 'sh...","[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...",0.0309752524,secondToLast,high,"[{'playerId': 'e6ok0deqkoe80184iu509gzu2', 'sh...",NaN,NaN,NaN,7.88,Pass


['Period start' 'Pass' 'Take On' 'Challenge' 'Blocked Pass'
 'Ball recovery' 'Attempted Tackle' 'Out' 'Ball touch' '50/50'
 'Dispossessed' 'Tackle' 'Corner Awarded' 'Clearance' 'Offside Pass'
 'Offside provoked' 'Foul' 'Aerial' 'Keeper pick-up' 'Deleted event'
 'Interception' 'Error' 'Goal' 'Attempt Saved' 'Save' 'Miss' 'Claim'
 'Card' 'Start delay' 'End delay' 'Referee Drop Ball' nan 'End'
 'Player Off' 'Player on' 'Formation change' 'Keeper Sweeper'
 'Shield ball opp']


,id,qualifier
2,2423549045,"{'id': 3586084711, 'qualifierId': 127, 'value'..."
3,2423549041,"{'id': 3586084701, 'qualifierId': 127, 'value'..."
4,2423549063,"{'id': 3586084825, 'qualifierId': 56, 'value':..."
4,2423549063,"{'id': 3586084833, 'qualifierId': 213, 'value'..."
4,2423549063,"{'id': 3586084827, 'qualifierId': 140, 'value'..."


...

,id,qualifierId,value
0,3586084711,127,Right to Left
1,3586084701,127,Left to Right
2,3586084825,56,Back
3,3586084833,213,2.7
4,3586084827,140,28.5


------------


,id,qualifierId,value,event_id
0,3586084711,127,Right to Left,2423549045
1,3586084701,127,Left to Right,2423549041
2,3586084825,56,Back,2423549063
3,3586084833,213,2.7,2423549063
4,3586084827,140,28.5,2423549063


------------


,id,qualifierId,value,event_id,qualifier
0,3586084711,127,Right to Left,2423549045,Direction of Play
1,3586084701,127,Left to Right,2423549041,Direction of Play
2,3586084825,56,Back,2423549063,Zone
3,3586084833,213,2.7,2423549063,Angle
4,3586084827,140,28.5,2423549063,Pass End X


In [43]:
# get the total minutes of the match
#df_tracking.groupby('current_phase')['timeelapsed'].max().reset_index()

df_minutes = df_tracking.groupby('player_id').size().reset_index(name='counts')

# fem sumatori i ho dividim entre 60 = 94.62 min!

# max counts = 141928, per tant, 141928/94.62  = 1500 per tant 1500 counts son 1 minut.

df_minutes['minutes'] = df_minutes['counts'] / 1500

df_minutes.drop('counts',axis=1,inplace=True)
df_minutes = df_minutes[df_minutes['player_id'] != 'aaaaaaaaaaaaaaaaaaaaaaaaa']

In [44]:
df_pos = df_tracking[['player_id','pos_x','pos_y','dop']]

# All with the same direction
df_pos['pos_x'] = pd.to_numeric(df_pos['pos_x'])
mask = df_pos['dop'] == 'R'
df_pos.loc[mask, 'dop'] = 'L'
df_pos.loc[mask, 'pos_x'] *= -1
df_pos.loc[mask, 'pos_y'] *= -1

# Divide the field into zones

# Into two zones
df_pos['ofensive'] = np.where(df_pos['pos_x'] > 0, 1, 0)
df_pos['defensive'] = np.where(df_pos['pos_x'] <= 0, 1, 0)

# More especific
df_pos['left_wing'] = np.where((df_pos['pos_x'] > 0) & (-34 <= df_pos['pos_y']) & (df_pos['pos_y'] < -11), 1, 0)
df_pos['right_wing'] = np.where((df_pos['pos_x'] > 0) & (11 < df_pos['pos_y']) & (df_pos['pos_y'] <= 34), 1, 0)
df_pos['center_down'] = np.where((df_pos['pos_x'] > 0) & (df_pos['pos_x'] <= 26.25) & (-11 <= df_pos['pos_y']) & (df_pos['pos_y'] <= 11), 1, 0)
df_pos['center_up'] = np.where((26.25 < df_pos['pos_x']) & (df_pos['pos_x'] <= 52.5) & (-11 <= df_pos['pos_y']) & (df_pos['pos_y'] <= 11), 1, 0)

pos_total = df_pos.groupby('player_id').agg({'ofensive': 'sum', 'defensive': 'sum',
                                             'left_wing': 'sum','right_wing': 'sum',
                                             'center_up': 'sum','center_down': 'sum'}).reset_index()

pos_total = pd.merge(pos_total, df_pos.groupby('player_id').size().reset_index(name='total'),
                      on='player_id', how='inner')

pos_total['ofensive %'] = pos_total['ofensive'] / pos_total['total']
pos_total['defensive %'] = pos_total['defensive'] / pos_total['total']
pos_total['left_wing %'] = pos_total['left_wing'] / pos_total['total']
pos_total['right_wing %'] = pos_total['right_wing'] / pos_total['total']
pos_total['center_up %'] = pos_total['center_up'] / pos_total['total']
pos_total['center_down %'] = pos_total['center_down'] / pos_total['total']

pos_pct_total = pos_total[['player_id','ofensive %','defensive %','left_wing %','right_wing %','center_up %','center_down %']]

#display(pos_pct_total)

pos_pct_total = pos_pct_total[pos_pct_total['ofensive %']>0.5]

df_minutes = pd.merge(df_minutes, pos_pct_total, on='player_id', how='right')
display(df_minutes)

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_8320\2680750106.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pos['pos_x'] = pd.to_numeric(df_pos['pos_x'])
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_8320\2680750106.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pos['ofensive'] = np.where(df_pos['pos_x'] > 0, 1, 0)
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_8320\2680750106.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

,player_id,minutes,ofensive %,defensive %,left_wing %,right_wing %,center_up %,center_down %
0,2lvit204llltk13iglsa2tjah,18.538667,0.506869,0.493131,0.027438,0.096195,0.069836,0.313399
1,3vx94h32ahujciraspdayj9t6,62.395333,0.572383,0.427617,0.159926,0.149573,0.095605,0.164457
2,5ak9fwtqlr2pll0nsv5br7p7u,38.255333,0.757350,0.242650,0.256330,0.230120,0.126623,0.135301
3,5qgc6zjc38a5xjl35gs7h3vu1,68.112000,0.757263,0.242737,0.095451,0.368990,0.111855,0.167831
4,6j0ogojh2b7poyceg7i3k09yi,94.618667,0.553196,0.446804,0.133638,0.089588,0.095605,0.232667
5,6u2ob6fv950r1qve8uejkq2uh,94.618667,0.749465,0.250535,0.281601,0.158418,0.126191,0.181071
6,7k0r5crdh9blj3edt31zwy0dm,26.505333,0.813145,0.186855,0.182077,0.088963,0.294129,0.247623
7,7ty1wdxxnusgkl34k5raipbl5,10.188667,0.800890,0.199110,0.449846,0.000000,0.137211,0.210168
8,agwvouyocx93y39g7tmwaojx1,32.222000,0.511824,0.488176,0.061738,0.108704,0.060766,0.280574
9,dxb1r4gqgxkngb0pzvfby9iol,38.122000,0.553434,0.446566,0.075425,0.109718,0.101429,0.266863


In [45]:
df_pos = df_tracking[['player_id','pos_x','pos_y','dop','timeelapsed']]

# All with the same direction
df_pos['pos_x'] = pd.to_numeric(df_pos['pos_x'])
mask = df_pos['dop'] == 'R'
df_pos.loc[mask, 'dop'] = 'L'
df_pos.loc[mask, 'pos_x'] *= -1

# Get the distance of each pair of points
df_pos = df_pos.sort_values(by=['player_id', 'timeelapsed'])
df_pos['delta_x'] = df_pos.groupby('player_id')['pos_x'].diff()
df_pos['delta_y'] = df_pos.groupby('player_id')['pos_y'].diff()
df_pos['distance'] = np.sqrt(df_pos['delta_x']**2 + df_pos['delta_y']**2)
df_pos['distance'] = df_pos['distance'].fillna(0)

df_pos['distance_covered'] = df_pos.groupby('player_id')['distance'].cumsum()

# Max distance covered per player_id
dist_cov = df_pos.groupby('player_id')['distance_covered'].max().reset_index()

df_minutes = pd.merge(df_minutes, dist_cov, on='player_id', how='left')

df_minutes['dist_cov_90'] = df_minutes['distance_covered'] / df_minutes['minutes'] * 90
df_minutes.drop('distance_covered',axis=1,inplace=True)
display(df_minutes)

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_8320\3662359512.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pos['pos_x'] = pd.to_numeric(df_pos['pos_x'])


,player_id,minutes,ofensive %,defensive %,left_wing %,right_wing %,center_up %,center_down %,dist_cov_90
0,2lvit204llltk13iglsa2tjah,18.538667,0.506869,0.493131,0.027438,0.096195,0.069836,0.313399,1.027953e+04
1,3vx94h32ahujciraspdayj9t6,62.395333,0.572383,0.427617,0.159926,0.149573,0.095605,0.164457,2.543756e+06
2,5ak9fwtqlr2pll0nsv5br7p7u,38.255333,0.757350,0.242650,0.256330,0.230120,0.126623,0.135301,1.200449e+04
3,5qgc6zjc38a5xjl35gs7h3vu1,68.112000,0.757263,0.242737,0.095451,0.368990,0.111855,0.167831,3.032644e+06
4,6j0ogojh2b7poyceg7i3k09yi,94.618667,0.553196,0.446804,0.133638,0.089588,0.095605,0.232667,4.031784e+06
5,6u2ob6fv950r1qve8uejkq2uh,94.618667,0.749465,0.250535,0.281601,0.158418,0.126191,0.181071,4.439340e+06
6,7k0r5crdh9blj3edt31zwy0dm,26.505333,0.813145,0.186855,0.182077,0.088963,0.294129,0.247623,1.004682e+04
7,7ty1wdxxnusgkl34k5raipbl5,10.188667,0.800890,0.199110,0.449846,0.000000,0.137211,0.210168,1.265141e+04
8,agwvouyocx93y39g7tmwaojx1,32.222000,0.511824,0.488176,0.061738,0.108704,0.060766,0.280574,1.015225e+04
9,dxb1r4gqgxkngb0pzvfby9iol,38.122000,0.553434,0.446566,0.075425,0.109718,0.101429,0.266863,1.130240e+04


In [46]:
acc_vel = df_tracking.groupby('player_id').agg({'acc': ['mean', 'max'], 'speed': ['mean', 'max']}).reset_index()
acc_vel.columns = ['player_id', 'acc_mean', 'acc_max', 'speed_mean', 'speed_max']

df_minutes = pd.merge(df_minutes, acc_vel, on='player_id', how='left')
display(df_minutes)

,player_id,minutes,ofensive %,defensive %,left_wing %,right_wing %,center_up %,center_down %,dist_cov_90,acc_mean,acc_max,speed_mean,speed_max
0,2lvit204llltk13iglsa2tjah,18.538667,0.506869,0.493131,0.027438,0.096195,0.069836,0.313399,1.027953e+04,-0.001197,5.044720,1.903511,7.592074
1,3vx94h32ahujciraspdayj9t6,62.395333,0.572383,0.427617,0.159926,0.149573,0.095605,0.164457,2.543756e+06,0.000241,6.538401,2.328649,9.112478
2,5ak9fwtqlr2pll0nsv5br7p7u,38.255333,0.757350,0.242650,0.256330,0.230120,0.126623,0.135301,1.200449e+04,-0.000514,7.038541,2.223043,8.654497
3,5qgc6zjc38a5xjl35gs7h3vu1,68.112000,0.757263,0.242737,0.095451,0.368990,0.111855,0.167831,3.032644e+06,-0.000415,7.981123,1.996893,9.851820
4,6j0ogojh2b7poyceg7i3k09yi,94.618667,0.553196,0.446804,0.133638,0.089588,0.095605,0.232667,4.031784e+06,0.000683,7.717428,2.070723,9.361088
5,6u2ob6fv950r1qve8uejkq2uh,94.618667,0.749465,0.250535,0.281601,0.158418,0.126191,0.181071,4.439340e+06,0.000336,6.782228,2.030733,9.344707
6,7k0r5crdh9blj3edt31zwy0dm,26.505333,0.813145,0.186855,0.182077,0.088963,0.294129,0.247623,1.004682e+04,-0.001370,7.242564,1.860533,7.616871
7,7ty1wdxxnusgkl34k5raipbl5,10.188667,0.800890,0.199110,0.449846,0.000000,0.137211,0.210168,1.265141e+04,0.000374,6.380166,2.342313,9.960378
8,agwvouyocx93y39g7tmwaojx1,32.222000,0.511824,0.488176,0.061738,0.108704,0.060766,0.280574,1.015225e+04,-0.000419,6.949152,1.879964,8.824575
9,dxb1r4gqgxkngb0pzvfby9iol,38.122000,0.553434,0.446566,0.075425,0.109718,0.101429,0.266863,1.130240e+04,0.000053,6.937418,2.093001,8.186618


In [48]:
def event_per_player(df,event):
    df = df[df['playerId'].notna()]
    try:
        df = df[df['event_description']==event]
        return df.groupby('playerId').size().reset_index(name=event)
    except:
        return False

df = event_per_player(df_events,'Pass')
df.rename(columns={'playerId': 'player_id'}, inplace=True)

df_minutes = pd.merge(df_minutes, df, on='player_id', how='left')
df_minutes['n_pass_90'] = df_minutes['Pass'] / df_minutes['minutes'] * 90
df_minutes.drop('Pass',axis=1,inplace=True)
display(df_minutes)

,player_id,minutes,ofensive %,defensive %,left_wing %,right_wing %,center_up %,center_down %,dist_cov_90,acc_mean,acc_max,speed_mean,speed_max,n_pass_90
0,2lvit204llltk13iglsa2tjah,18.538667,0.506869,0.493131,0.027438,0.096195,0.069836,0.313399,1.027953e+04,-0.001197,5.044720,1.903511,7.592074,9.709436
1,3vx94h32ahujciraspdayj9t6,62.395333,0.572383,0.427617,0.159926,0.149573,0.095605,0.164457,2.543756e+06,0.000241,6.538401,2.328649,9.112478,27.405896
2,5ak9fwtqlr2pll0nsv5br7p7u,38.255333,0.757350,0.242650,0.256330,0.230120,0.126623,0.135301,1.200449e+04,-0.000514,7.038541,2.223043,8.654497,47.052263
3,5qgc6zjc38a5xjl35gs7h3vu1,68.112000,0.757263,0.242737,0.095451,0.368990,0.111855,0.167831,3.032644e+06,-0.000415,7.981123,1.996893,9.851820,47.568710
4,6j0ogojh2b7poyceg7i3k09yi,94.618667,0.553196,0.446804,0.133638,0.089588,0.095605,0.232667,4.031784e+06,0.000683,7.717428,2.070723,9.361088,60.875937
5,6u2ob6fv950r1qve8uejkq2uh,94.618667,0.749465,0.250535,0.281601,0.158418,0.126191,0.181071,4.439340e+06,0.000336,6.782228,2.030733,9.344707,56.120005
6,7k0r5crdh9blj3edt31zwy0dm,26.505333,0.813145,0.186855,0.182077,0.088963,0.294129,0.247623,1.004682e+04,-0.001370,7.242564,1.860533,7.616871,3.395543
7,7ty1wdxxnusgkl34k5raipbl5,10.188667,0.800890,0.199110,0.449846,0.000000,0.137211,0.210168,1.265141e+04,0.000374,6.380166,2.342313,9.960378,NaN
8,agwvouyocx93y39g7tmwaojx1,32.222000,0.511824,0.488176,0.061738,0.108704,0.060766,0.280574,1.015225e+04,-0.000419,6.949152,1.879964,8.824575,13.965614
9,dxb1r4gqgxkngb0pzvfby9iol,38.122000,0.553434,0.446566,0.075425,0.109718,0.101429,0.266863,1.130240e+04,0.000053,6.937418,2.093001,8.186618,30.690940


In [49]:
df_passes = df_events[df_events['event_description']=='Pass']
df_1=df_passes[df_passes['assist']==True].groupby('playerId').size().reset_index(name='assist')
df_2=df_passes[df_passes['keyPass']==True].groupby('playerId').size().reset_index(name='keyPass')

def concat_df(df_minutes,df,column_name):
    df.rename(columns={'playerId': 'player_id'}, inplace=True)
    df_minutes = pd.merge(df_minutes, df, on='player_id', how='left')
    df_minutes[str(column_name)+'_90'] = df_minutes[column_name] / df_minutes['minutes'] * 90
    df_minutes.drop(str(column_name),axis=1,inplace=True)
    return df_minutes

df_minutes = concat_df(df_minutes,df_1,'assist')
df_minutes = concat_df(df_minutes,df_2,'keyPass')
display(df_minutes)

,player_id,minutes,ofensive %,defensive %,left_wing %,right_wing %,center_up %,center_down %,dist_cov_90,acc_mean,acc_max,speed_mean,speed_max,n_pass_90,assist_90,keyPass_90
0,2lvit204llltk13iglsa2tjah,18.538667,0.506869,0.493131,0.027438,0.096195,0.069836,0.313399,1.027953e+04,-0.001197,5.044720,1.903511,7.592074,9.709436,NaN,NaN
1,3vx94h32ahujciraspdayj9t6,62.395333,0.572383,0.427617,0.159926,0.149573,0.095605,0.164457,2.543756e+06,0.000241,6.538401,2.328649,9.112478,27.405896,NaN,NaN
2,5ak9fwtqlr2pll0nsv5br7p7u,38.255333,0.757350,0.242650,0.256330,0.230120,0.126623,0.135301,1.200449e+04,-0.000514,7.038541,2.223043,8.654497,47.052263,NaN,2.352613
3,5qgc6zjc38a5xjl35gs7h3vu1,68.112000,0.757263,0.242737,0.095451,0.368990,0.111855,0.167831,3.032644e+06,-0.000415,7.981123,1.996893,9.851820,47.568710,NaN,NaN
4,6j0ogojh2b7poyceg7i3k09yi,94.618667,0.553196,0.446804,0.133638,0.089588,0.095605,0.232667,4.031784e+06,0.000683,7.717428,2.070723,9.361088,60.875937,NaN,0.951187
5,6u2ob6fv950r1qve8uejkq2uh,94.618667,0.749465,0.250535,0.281601,0.158418,0.126191,0.181071,4.439340e+06,0.000336,6.782228,2.030733,9.344707,56.120005,0.951187,0.951187
6,7k0r5crdh9blj3edt31zwy0dm,26.505333,0.813145,0.186855,0.182077,0.088963,0.294129,0.247623,1.004682e+04,-0.001370,7.242564,1.860533,7.616871,3.395543,NaN,NaN
7,7ty1wdxxnusgkl34k5raipbl5,10.188667,0.800890,0.199110,0.449846,0.000000,0.137211,0.210168,1.265141e+04,0.000374,6.380166,2.342313,9.960378,NaN,NaN,NaN
8,agwvouyocx93y39g7tmwaojx1,32.222000,0.511824,0.488176,0.061738,0.108704,0.060766,0.280574,1.015225e+04,-0.000419,6.949152,1.879964,8.824575,13.965614,NaN,NaN
9,dxb1r4gqgxkngb0pzvfby9iol,38.122000,0.553434,0.446566,0.075425,0.109718,0.101429,0.266863,1.130240e+04,0.000053,6.937418,2.093001,8.186618,30.690940,NaN,NaN


In [50]:
xthreat = df_events[df_events['xThreat.applied'].notna()]
print(xthreat['event_description'].value_counts())

xthreat = xthreat[xthreat['playerId'].notna()]
xthreat['xThreat.applied'] = pd.to_numeric(xthreat['xThreat.applied'])
xthreat = xthreat.groupby('playerId')['xThreat.applied'].mean().reset_index(name='xthreat')

xthreat.rename(columns={'playerId': 'player_id'}, inplace=True)
df_minutes = pd.merge(df_minutes, xthreat, on='player_id', how='left')
display(df_minutes)

Pass            763
Offside Pass      1
Name: event_description, dtype: int64


,player_id,minutes,ofensive %,defensive %,left_wing %,right_wing %,center_up %,center_down %,dist_cov_90,acc_mean,acc_max,speed_mean,speed_max,n_pass_90,assist_90,keyPass_90,xthreat
0,2lvit204llltk13iglsa2tjah,18.538667,0.506869,0.493131,0.027438,0.096195,0.069836,0.313399,1.027953e+04,-0.001197,5.044720,1.903511,7.592074,9.709436,NaN,NaN,NaN
1,3vx94h32ahujciraspdayj9t6,62.395333,0.572383,0.427617,0.159926,0.149573,0.095605,0.164457,2.543756e+06,0.000241,6.538401,2.328649,9.112478,27.405896,NaN,NaN,0.027412
2,5ak9fwtqlr2pll0nsv5br7p7u,38.255333,0.757350,0.242650,0.256330,0.230120,0.126623,0.135301,1.200449e+04,-0.000514,7.038541,2.223043,8.654497,47.052263,NaN,2.352613,0.099773
3,5qgc6zjc38a5xjl35gs7h3vu1,68.112000,0.757263,0.242737,0.095451,0.368990,0.111855,0.167831,3.032644e+06,-0.000415,7.981123,1.996893,9.851820,47.568710,NaN,NaN,0.079592
4,6j0ogojh2b7poyceg7i3k09yi,94.618667,0.553196,0.446804,0.133638,0.089588,0.095605,0.232667,4.031784e+06,0.000683,7.717428,2.070723,9.361088,60.875937,NaN,0.951187,0.046613
5,6u2ob6fv950r1qve8uejkq2uh,94.618667,0.749465,0.250535,0.281601,0.158418,0.126191,0.181071,4.439340e+06,0.000336,6.782228,2.030733,9.344707,56.120005,0.951187,0.951187,0.095785
6,7k0r5crdh9blj3edt31zwy0dm,26.505333,0.813145,0.186855,0.182077,0.088963,0.294129,0.247623,1.004682e+04,-0.001370,7.242564,1.860533,7.616871,3.395543,NaN,NaN,NaN
7,7ty1wdxxnusgkl34k5raipbl5,10.188667,0.800890,0.199110,0.449846,0.000000,0.137211,0.210168,1.265141e+04,0.000374,6.380166,2.342313,9.960378,NaN,NaN,NaN,NaN
8,agwvouyocx93y39g7tmwaojx1,32.222000,0.511824,0.488176,0.061738,0.108704,0.060766,0.280574,1.015225e+04,-0.000419,6.949152,1.879964,8.824575,13.965614,NaN,NaN,0.068901
9,dxb1r4gqgxkngb0pzvfby9iol,38.122000,0.553434,0.446566,0.075425,0.109718,0.101429,0.266863,1.130240e+04,0.000053,6.937418,2.093001,8.186618,30.690940,NaN,NaN,0.037555


In [54]:
df_passes = df_events[df_events['event_description']=='Pass']

# ---------- MEAN DISTANCE PER PLAYER --------------------
df_completo = pd.merge(df_passes, qualifiers[qualifiers['qualifier'] == 'Length'], left_on = 'id',right_on='event_id')
df_completo['value'] = pd.to_numeric(df_completo['value'])
df = df_completo.groupby('playerId')['value'].mean().reset_index(name='mean_pass_distance')

df.rename(columns={'playerId': 'player_id'}, inplace=True)
#df_minutes = pd.merge(df_minutes, df, on='player_id', how='left')

# ---------- TOTAL SHORT AND LONG PASSES PER PLAYER --------------------

df_completo['long'] = df_completo['value']>25

count = df_completo.groupby('playerId')['long'].value_counts().unstack(fill_value=0)
count.reset_index(inplace=True)

count.columns = ['player_id','Short passes', 'Long passes']

df_minutes = pd.merge(df_minutes, count, on='player_id', how='left')
df_minutes['short_passes_90'] = df_minutes['Short passes'] / df_minutes['minutes'] * 90
df_minutes.drop('Short passes',axis=1,inplace=True)
df_minutes['long_passes_90'] = df_minutes['Long passes'] / df_minutes['minutes'] * 90
df_minutes.drop('Long passes',axis=1,inplace=True)
display(df_minutes)


,player_id,minutes,ofensive %,defensive %,left_wing %,right_wing %,center_up %,center_down %,dist_cov_90,acc_mean,acc_max,speed_mean,speed_max,n_pass_90,assist_90,keyPass_90,xthreat,mean_pass_distance,short_passes_90,long_passes_90
0,2lvit204llltk13iglsa2tjah,18.538667,0.506869,0.493131,0.027438,0.096195,0.069836,0.313399,1.027953e+04,-0.001197,5.044720,1.903511,7.592074,9.709436,NaN,NaN,NaN,24.800000,4.854718,4.854718
1,3vx94h32ahujciraspdayj9t6,62.395333,0.572383,0.427617,0.159926,0.149573,0.095605,0.164457,2.543756e+06,0.000241,6.538401,2.328649,9.112478,27.405896,NaN,NaN,0.027412,12.226316,25.963480,1.442416
2,5ak9fwtqlr2pll0nsv5br7p7u,38.255333,0.757350,0.242650,0.256330,0.230120,0.126623,0.135301,1.200449e+04,-0.000514,7.038541,2.223043,8.654497,47.052263,NaN,2.352613,0.099773,18.615000,32.936584,14.115679
3,5qgc6zjc38a5xjl35gs7h3vu1,68.112000,0.757263,0.242737,0.095451,0.368990,0.111855,0.167831,3.032644e+06,-0.000415,7.981123,1.996893,9.851820,47.568710,NaN,NaN,0.079592,18.286111,36.997886,10.570825
4,6j0ogojh2b7poyceg7i3k09yi,94.618667,0.553196,0.446804,0.133638,0.089588,0.095605,0.232667,4.031784e+06,0.000683,7.717428,2.070723,9.361088,60.875937,NaN,0.951187,0.046613,15.412500,52.315258,8.560679
5,6u2ob6fv950r1qve8uejkq2uh,94.618667,0.749465,0.250535,0.281601,0.158418,0.126191,0.181071,4.439340e+06,0.000336,6.782228,2.030733,9.344707,56.120005,0.951187,0.951187,0.095785,16.323729,46.608139,9.511865
6,7k0r5crdh9blj3edt31zwy0dm,26.505333,0.813145,0.186855,0.182077,0.088963,0.294129,0.247623,1.004682e+04,-0.001370,7.242564,1.860533,7.616871,3.395543,NaN,NaN,NaN,20.200000,3.395543,0.000000
7,7ty1wdxxnusgkl34k5raipbl5,10.188667,0.800890,0.199110,0.449846,0.000000,0.137211,0.210168,1.265141e+04,0.000374,6.380166,2.342313,9.960378,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,agwvouyocx93y39g7tmwaojx1,32.222000,0.511824,0.488176,0.061738,0.108704,0.060766,0.280574,1.015225e+04,-0.000419,6.949152,1.879964,8.824575,13.965614,NaN,NaN,0.068901,22.740000,5.586245,8.379368
9,dxb1r4gqgxkngb0pzvfby9iol,38.122000,0.553434,0.446566,0.075425,0.109718,0.101429,0.266863,1.130240e+04,0.000053,6.937418,2.093001,8.186618,30.690940,NaN,NaN,0.037555,15.138462,28.330098,2.360842


In [55]:
df_shots = df_events[df_events['event_description'].isin(['Goal','Miss'])]

df_shots = df_shots.groupby('playerId').size().reset_index(name='shots')

df_minutes=concat_df(df_minutes,df_shots,'shots')
display(df_minutes)

,player_id,minutes,ofensive %,defensive %,left_wing %,right_wing %,center_up %,center_down %,dist_cov_90,acc_mean,acc_max,speed_mean,speed_max,n_pass_90,assist_90,keyPass_90,xthreat,mean_pass_distance,short_passes_90,long_passes_90,shots_90
0,2lvit204llltk13iglsa2tjah,18.538667,0.506869,0.493131,0.027438,0.096195,0.069836,0.313399,1.027953e+04,-0.001197,5.044720,1.903511,7.592074,9.709436,NaN,NaN,NaN,24.800000,4.854718,4.854718,NaN
1,3vx94h32ahujciraspdayj9t6,62.395333,0.572383,0.427617,0.159926,0.149573,0.095605,0.164457,2.543756e+06,0.000241,6.538401,2.328649,9.112478,27.405896,NaN,NaN,0.027412,12.226316,25.963480,1.442416,1.442416
2,5ak9fwtqlr2pll0nsv5br7p7u,38.255333,0.757350,0.242650,0.256330,0.230120,0.126623,0.135301,1.200449e+04,-0.000514,7.038541,2.223043,8.654497,47.052263,NaN,2.352613,0.099773,18.615000,32.936584,14.115679,NaN
3,5qgc6zjc38a5xjl35gs7h3vu1,68.112000,0.757263,0.242737,0.095451,0.368990,0.111855,0.167831,3.032644e+06,-0.000415,7.981123,1.996893,9.851820,47.568710,NaN,NaN,0.079592,18.286111,36.997886,10.570825,NaN
4,6j0ogojh2b7poyceg7i3k09yi,94.618667,0.553196,0.446804,0.133638,0.089588,0.095605,0.232667,4.031784e+06,0.000683,7.717428,2.070723,9.361088,60.875937,NaN,0.951187,0.046613,15.412500,52.315258,8.560679,0.951187
5,6u2ob6fv950r1qve8uejkq2uh,94.618667,0.749465,0.250535,0.281601,0.158418,0.126191,0.181071,4.439340e+06,0.000336,6.782228,2.030733,9.344707,56.120005,0.951187,0.951187,0.095785,16.323729,46.608139,9.511865,0.951187
6,7k0r5crdh9blj3edt31zwy0dm,26.505333,0.813145,0.186855,0.182077,0.088963,0.294129,0.247623,1.004682e+04,-0.001370,7.242564,1.860533,7.616871,3.395543,NaN,NaN,NaN,20.200000,3.395543,0.000000,NaN
7,7ty1wdxxnusgkl34k5raipbl5,10.188667,0.800890,0.199110,0.449846,0.000000,0.137211,0.210168,1.265141e+04,0.000374,6.380166,2.342313,9.960378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,agwvouyocx93y39g7tmwaojx1,32.222000,0.511824,0.488176,0.061738,0.108704,0.060766,0.280574,1.015225e+04,-0.000419,6.949152,1.879964,8.824575,13.965614,NaN,NaN,0.068901,22.740000,5.586245,8.379368,NaN
9,dxb1r4gqgxkngb0pzvfby9iol,38.122000,0.553434,0.446566,0.075425,0.109718,0.101429,0.266863,1.130240e+04,0.000053,6.937418,2.093001,8.186618,30.690940,NaN,NaN,0.037555,15.138462,28.330098,2.360842,NaN


In [56]:
df_1 = df_events[(df_events['lineBreakingPass.linesBroken.value'].notna()) & (df_events['lineBreakingPass.linesBroken.value']!=0)].groupby('playerId').size().reset_index(name='line breaking passes')
df_2 = df_events[df_events['lineBreakingPass.lastLineBroken.value'].notna()].groupby('playerId').size().reset_index(name='last_line breaking passes')

df_minutes=concat_df(df_minutes,df_1,'line breaking passes')
df_minutes=concat_df(df_minutes,df_2,'last_line breaking passes')
display(df_minutes)

,player_id,minutes,ofensive %,defensive %,left_wing %,right_wing %,center_up %,center_down %,dist_cov_90,acc_mean,acc_max,speed_mean,speed_max,n_pass_90,assist_90,keyPass_90,xthreat,mean_pass_distance,short_passes_90,long_passes_90,shots_90,line breaking passes_90,last_line breaking passes_90
0,2lvit204llltk13iglsa2tjah,18.538667,0.506869,0.493131,0.027438,0.096195,0.069836,0.313399,1.027953e+04,-0.001197,5.044720,1.903511,7.592074,9.709436,NaN,NaN,NaN,24.800000,4.854718,4.854718,NaN,NaN,NaN
1,3vx94h32ahujciraspdayj9t6,62.395333,0.572383,0.427617,0.159926,0.149573,0.095605,0.164457,2.543756e+06,0.000241,6.538401,2.328649,9.112478,27.405896,NaN,NaN,0.027412,12.226316,25.963480,1.442416,1.442416,5.769662,2.884831
2,5ak9fwtqlr2pll0nsv5br7p7u,38.255333,0.757350,0.242650,0.256330,0.230120,0.126623,0.135301,1.200449e+04,-0.000514,7.038541,2.223043,8.654497,47.052263,NaN,2.352613,0.099773,18.615000,32.936584,14.115679,NaN,9.410453,4.705226
3,5qgc6zjc38a5xjl35gs7h3vu1,68.112000,0.757263,0.242737,0.095451,0.368990,0.111855,0.167831,3.032644e+06,-0.000415,7.981123,1.996893,9.851820,47.568710,NaN,NaN,0.079592,18.286111,36.997886,10.570825,NaN,10.570825,NaN
4,6j0ogojh2b7poyceg7i3k09yi,94.618667,0.553196,0.446804,0.133638,0.089588,0.095605,0.232667,4.031784e+06,0.000683,7.717428,2.070723,9.361088,60.875937,NaN,0.951187,0.046613,15.412500,52.315258,8.560679,0.951187,26.633222,6.658306
5,6u2ob6fv950r1qve8uejkq2uh,94.618667,0.749465,0.250535,0.281601,0.158418,0.126191,0.181071,4.439340e+06,0.000336,6.782228,2.030733,9.344707,56.120005,0.951187,0.951187,0.095785,16.323729,46.608139,9.511865,0.951187,19.023730,7.609492
6,7k0r5crdh9blj3edt31zwy0dm,26.505333,0.813145,0.186855,0.182077,0.088963,0.294129,0.247623,1.004682e+04,-0.001370,7.242564,1.860533,7.616871,3.395543,NaN,NaN,NaN,20.200000,3.395543,0.000000,NaN,NaN,NaN
7,7ty1wdxxnusgkl34k5raipbl5,10.188667,0.800890,0.199110,0.449846,0.000000,0.137211,0.210168,1.265141e+04,0.000374,6.380166,2.342313,9.960378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,agwvouyocx93y39g7tmwaojx1,32.222000,0.511824,0.488176,0.061738,0.108704,0.060766,0.280574,1.015225e+04,-0.000419,6.949152,1.879964,8.824575,13.965614,NaN,NaN,0.068901,22.740000,5.586245,8.379368,NaN,2.793123,NaN
9,dxb1r4gqgxkngb0pzvfby9iol,38.122000,0.553434,0.446566,0.075425,0.109718,0.101429,0.266863,1.130240e+04,0.000053,6.937418,2.093001,8.186618,30.690940,NaN,NaN,0.037555,15.138462,28.330098,2.360842,NaN,11.804208,4.721683


In [57]:
df_all = pd.merge(df_events,df_tracking, on =['timeelapsed','current_phase'])

df_shots = df_all[(df_all['event_description'].isin(['Goal','Miss'])) & (df_all['player_id']==df_all['playerId'])]

df_shots['pos_x'] = pd.to_numeric(df_shots['pos_x'])
mask = df_shots['dop'] == 'R'
df_shots.loc[mask, 'dop'] = 'L'
df_shots.loc[mask, 'pos_x'] *= -1

# determine the distance
target_point = (52.5, 0)

df_shots['distance_to_target'] = np.sqrt((df_shots['pos_x'] - target_point[0])**2 + (df_shots['pos_y'] - target_point[1])**2)

df_shots = df_shots.groupby('playerId')['distance_to_target'].mean().reset_index(name='avg_shot_distance')

df_shots.rename(columns={'playerId': 'player_id'}, inplace=True)
df_minutes = pd.merge(df_minutes, df_shots, on='player_id', how='left')
display(df_minutes)

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_8320\1365087383.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shots['pos_x'] = pd.to_numeric(df_shots['pos_x'])
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_8320\1365087383.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shots['distance_to_target'] = np.sqrt((df_shots['pos_x'] - target_point[0])**2 + (df_shots['pos_y'] - target_point[1])**2)


,player_id,minutes,ofensive %,defensive %,left_wing %,right_wing %,center_up %,center_down %,dist_cov_90,acc_mean,acc_max,speed_mean,speed_max,n_pass_90,assist_90,keyPass_90,xthreat,mean_pass_distance,short_passes_90,long_passes_90,shots_90,line breaking passes_90,last_line breaking passes_90,avg_shot_distance
0,2lvit204llltk13iglsa2tjah,18.538667,0.506869,0.493131,0.027438,0.096195,0.069836,0.313399,1.027953e+04,-0.001197,5.044720,1.903511,7.592074,9.709436,NaN,NaN,NaN,24.800000,4.854718,4.854718,NaN,NaN,NaN,NaN
1,3vx94h32ahujciraspdayj9t6,62.395333,0.572383,0.427617,0.159926,0.149573,0.095605,0.164457,2.543756e+06,0.000241,6.538401,2.328649,9.112478,27.405896,NaN,NaN,0.027412,12.226316,25.963480,1.442416,1.442416,5.769662,2.884831,20.554468
2,5ak9fwtqlr2pll0nsv5br7p7u,38.255333,0.757350,0.242650,0.256330,0.230120,0.126623,0.135301,1.200449e+04,-0.000514,7.038541,2.223043,8.654497,47.052263,NaN,2.352613,0.099773,18.615000,32.936584,14.115679,NaN,9.410453,4.705226,NaN
3,5qgc6zjc38a5xjl35gs7h3vu1,68.112000,0.757263,0.242737,0.095451,0.368990,0.111855,0.167831,3.032644e+06,-0.000415,7.981123,1.996893,9.851820,47.568710,NaN,NaN,0.079592,18.286111,36.997886,10.570825,NaN,10.570825,NaN,NaN
4,6j0ogojh2b7poyceg7i3k09yi,94.618667,0.553196,0.446804,0.133638,0.089588,0.095605,0.232667,4.031784e+06,0.000683,7.717428,2.070723,9.361088,60.875937,NaN,0.951187,0.046613,15.412500,52.315258,8.560679,0.951187,26.633222,6.658306,27.086573
5,6u2ob6fv950r1qve8uejkq2uh,94.618667,0.749465,0.250535,0.281601,0.158418,0.126191,0.181071,4.439340e+06,0.000336,6.782228,2.030733,9.344707,56.120005,0.951187,0.951187,0.095785,16.323729,46.608139,9.511865,0.951187,19.023730,7.609492,24.234017
6,7k0r5crdh9blj3edt31zwy0dm,26.505333,0.813145,0.186855,0.182077,0.088963,0.294129,0.247623,1.004682e+04,-0.001370,7.242564,1.860533,7.616871,3.395543,NaN,NaN,NaN,20.200000,3.395543,0.000000,NaN,NaN,NaN,NaN
7,7ty1wdxxnusgkl34k5raipbl5,10.188667,0.800890,0.199110,0.449846,0.000000,0.137211,0.210168,1.265141e+04,0.000374,6.380166,2.342313,9.960378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,agwvouyocx93y39g7tmwaojx1,32.222000,0.511824,0.488176,0.061738,0.108704,0.060766,0.280574,1.015225e+04,-0.000419,6.949152,1.879964,8.824575,13.965614,NaN,NaN,0.068901,22.740000,5.586245,8.379368,NaN,2.793123,NaN,NaN
9,dxb1r4gqgxkngb0pzvfby9iol,38.122000,0.553434,0.446566,0.075425,0.109718,0.101429,0.266863,1.130240e+04,0.000053,6.937418,2.093001,8.186618,30.690940,NaN,NaN,0.037555,15.138462,28.330098,2.360842,NaN,11.804208,4.721683,NaN


In [58]:
df_events[df_events['event_description']=='Take On']['outcome'].value_counts()
df_takeons = df_events[df_events['event_description']=='Take On'].groupby('playerId').size().reset_index(name='take on')

df_minutes = concat_df(df_minutes,df_takeons,'take on')
display(df_minutes)

,player_id,minutes,ofensive %,defensive %,left_wing %,right_wing %,center_up %,center_down %,dist_cov_90,acc_mean,acc_max,speed_mean,speed_max,n_pass_90,assist_90,keyPass_90,xthreat,mean_pass_distance,short_passes_90,long_passes_90,shots_90,line breaking passes_90,last_line breaking passes_90,avg_shot_distance,take on_90
0,2lvit204llltk13iglsa2tjah,18.538667,0.506869,0.493131,0.027438,0.096195,0.069836,0.313399,1.027953e+04,-0.001197,5.044720,1.903511,7.592074,9.709436,NaN,NaN,NaN,24.800000,4.854718,4.854718,NaN,NaN,NaN,NaN,NaN
1,3vx94h32ahujciraspdayj9t6,62.395333,0.572383,0.427617,0.159926,0.149573,0.095605,0.164457,2.543756e+06,0.000241,6.538401,2.328649,9.112478,27.405896,NaN,NaN,0.027412,12.226316,25.963480,1.442416,1.442416,5.769662,2.884831,20.554468,5.769662
2,5ak9fwtqlr2pll0nsv5br7p7u,38.255333,0.757350,0.242650,0.256330,0.230120,0.126623,0.135301,1.200449e+04,-0.000514,7.038541,2.223043,8.654497,47.052263,NaN,2.352613,0.099773,18.615000,32.936584,14.115679,NaN,9.410453,4.705226,NaN,NaN
3,5qgc6zjc38a5xjl35gs7h3vu1,68.112000,0.757263,0.242737,0.095451,0.368990,0.111855,0.167831,3.032644e+06,-0.000415,7.981123,1.996893,9.851820,47.568710,NaN,NaN,0.079592,18.286111,36.997886,10.570825,NaN,10.570825,NaN,NaN,3.964059
4,6j0ogojh2b7poyceg7i3k09yi,94.618667,0.553196,0.446804,0.133638,0.089588,0.095605,0.232667,4.031784e+06,0.000683,7.717428,2.070723,9.361088,60.875937,NaN,0.951187,0.046613,15.412500,52.315258,8.560679,0.951187,26.633222,6.658306,27.086573,2.853560
5,6u2ob6fv950r1qve8uejkq2uh,94.618667,0.749465,0.250535,0.281601,0.158418,0.126191,0.181071,4.439340e+06,0.000336,6.782228,2.030733,9.344707,56.120005,0.951187,0.951187,0.095785,16.323729,46.608139,9.511865,0.951187,19.023730,7.609492,24.234017,3.804746
6,7k0r5crdh9blj3edt31zwy0dm,26.505333,0.813145,0.186855,0.182077,0.088963,0.294129,0.247623,1.004682e+04,-0.001370,7.242564,1.860533,7.616871,3.395543,NaN,NaN,NaN,20.200000,3.395543,0.000000,NaN,NaN,NaN,NaN,NaN
7,7ty1wdxxnusgkl34k5raipbl5,10.188667,0.800890,0.199110,0.449846,0.000000,0.137211,0.210168,1.265141e+04,0.000374,6.380166,2.342313,9.960378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,agwvouyocx93y39g7tmwaojx1,32.222000,0.511824,0.488176,0.061738,0.108704,0.060766,0.280574,1.015225e+04,-0.000419,6.949152,1.879964,8.824575,13.965614,NaN,NaN,0.068901,22.740000,5.586245,8.379368,NaN,2.793123,NaN,NaN,5.586245
9,dxb1r4gqgxkngb0pzvfby9iol,38.122000,0.553434,0.446566,0.075425,0.109718,0.101429,0.266863,1.130240e+04,0.000053,6.937418,2.093001,8.186618,30.690940,NaN,NaN,0.037555,15.138462,28.330098,2.360842,NaN,11.804208,4.721683,NaN,NaN
